In [132]:
import pickle
import itertools
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pathlib

In [133]:
## mask.csvように書き換える
class metashape_Camera:
    def __init__(self, val):
        self.img_num = val
        inner_para_path = pathlib.Path("C:/home/study/git_test/curve_sketch/real_data/view_mat/inner_para.csv")
        self.A = np.loadtxt(str(inner_para_path), delimiter=",")

    def img_load(self):
        folder_path = "C:/home/study/git_test/curve_sketch/real_data/GmJMC025_02"
        file_path = os.path.join(folder_path, str(self.img_num) + ".JPG")
        img = cv2.imread(file_path, 1)# BGRで読み込み
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #img = cv2.flip(img, 1) # 反転 unity用?
        self.img = img # 画像(ndarray)
        self.img_shape = img.shape

    def mask_load(self):
        folder = pathlib.Path("C:/home/study/git_test/curve_sketch/real_data/masks/annotation_mask")
        masks_path = folder.glob(str(self.img_num)+"_"+"*"+".jpg")

        mask_list = []
        for mask_path in masks_path:
            mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
            #mask = cood_to_mask(mask_path, (self.img_shape[0], self.img_shape[1]))
            #contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
            mask_list.append(mask)
        #print(np.asarray(mask_list).shape)
        self.masks = np.asarray(mask_list).transpose(1, 2, 0)
        
        
        
    def para_load(self):
        folder_path = pathlib.Path("C:/home/study/git_test/curve_sketch/real_data/view_mat/")
        file_path = os.path.join(folder_path, str(self.img_num)+".csv")
        self.Rt = np.loadtxt(file_path, delimiter=",")
        self.Rt = np.linalg.pinv(self.Rt)
        self.P = np.dot(self.A, self.Rt[0:3,0:4])
        self.cam_world_cood = -np.dot(self.Rt[0:3,0:3].T, self.Rt[0:3,3])
        
def cood_to_mask(csv_path, im_shape):
    idx = np.loadtxt(str(csv_path), delimiter=",")
    idx = idx.astype(np.int64)
    
    mask = np.zeros(im_shape, dtype=np.uint8)
    
    if idx.size == 0:
        return mask
    
    mask[idx[0],idx[1]] = 255
    return mask

def normalization(vec3):
    return np.array([vec3[:,0]/vec3[:,2], vec3[:,1]/vec3[:,2]]).T

In [134]:
def norm(vec):
    s_vec = np.square(vec)
    sume = np.sum(s_vec)
    ss = np.sqrt(sume)
    return vec/ss

In [135]:
pcd = o3d.io.read_point_cloud("polygon/GmJMC025_02_ds_denoise.ply")

#labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=100))
with open(r"domain_expansion/point_cloud_label.pickle", 'rb') as f:
    labels = pickle.load(f)

max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

LABELS = labels
PCD = pcd

class LabelProp:
    
    def __init__(self, val, pcd, labels):
        self.num = val
        self.LABELS = labels
        self.PCD = pcd
        
    def cam_load(self):
        test = metashape_Camera(self.num)
        test.img_load()
        test.mask_load()
        test.para_load()
        self.cam = test
    
    def point_cloud_load(self):
        #pcd = o3d.io.read_point_cloud("denoise.ply")

        #with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        #labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=100))

        #max_label = labels.max()
        #colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
        #colors[labels < 0] = 0
        #pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
        
        self.pcd = self.PCD
        self.labels = self.LABELS
    
    def pc_prop(self):
        # 点群加工
        camera = self.cam.cam_world_cood
        diameter = np.linalg.norm(
            np.asarray(self.pcd.get_max_bound()) - np.asarray(self.pcd.get_min_bound()))

        #camera = cam_world_cood
        radius = diameter*1000

        _, pt_map = self.pcd.hidden_point_removal(camera, radius)
        self.pcd = self.pcd.select_by_index(sorted(pt_map))

        self.TD_points = np.concatenate([np.asarray(self.pcd.points), np.ones((np.asarray(self.pcd.points).shape[0],1))], axis=1)
        self.labels = self.labels[np.array(sorted(pt_map))]
        
    def pc_labeling(self):
        repro_points = normalization((self.cam.P@self.TD_points.T).T)
        repro_points = np.round(repro_points).astype(np.int64)
        
        idx = (repro_points[:,0]>0) & (repro_points[:,0]<self.cam.img_shape[1]) & (repro_points[:,1]>0) & (repro_points[:,1]<self.cam.img_shape[0])
        
        self.repro_points = repro_points[idx]
        self.labels = self.labels[idx]
        
    def labeling(self):
        overrap_list = np.zeros((labels.max().astype(int)+1 , self.cam.masks.shape[2]))
        for j in np.unique(self.labels):
            temp_overrap_list = []
            zero_mask = np.zeros([self.cam.masks.shape[0],self.cam.masks.shape[1]])
            zero_mask[self.repro_points[self.labels==j][:,1], self.repro_points[self.labels==j][:,0]] = 1
            for i in range(self.cam.masks.shape[2]):
                temp_overrap_list.append(np.sum((zero_mask>0) * (self.cam.masks[:,:,i]>0)))
            overrap_list[int(j),:] = np.array(temp_overrap_list)
        self.output_label = np.argmax(overrap_list,axis=0)

def prop_all(num, pcd, labels):
    lp = LabelProp(num, pcd, labels)
    lp.cam_load()
    lp.point_cloud_load()
    lp.pc_prop()
    lp.pc_labeling()
    lp.labeling()
    return lp.output_label

In [136]:
l = []
for i in range(0,264):
    label = prop_all(i,PCD,LABELS)
    l.append(label)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


KeyboardInterrupt: 

In [ ]:
with open('label.pickle', 'wb') as f:
    pickle.dump(l, f)